In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Enable mixed precision
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

# Enable XLA for GPU optimization
tf.config.optimizer.set_jit(True)

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Paths for dataset
CSV_PATH = "../Dataset/image_labels.csv"
IMG_DIR = "../Dataset/interior"
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
EPOCHS = 140

# Generate a CSV file mapping image paths to their labels
def regenerate_csv(image_dir, output_file):
    classes = ['bath', 'bed', 'dining room', 'kitchen', 'living room']
    class_variations = {
        'bath': ['bath', 'bathroom'], 'bed': ['bed', 'bedroom'],
        'dining room': ['dining', 'dining_room', 'diningroom', 'din'],
        'kitchen': ['kitchen'], 'living room': ['living', 'living_room', 'livingroom']
    }
    data = []
    for filename in os.listdir(image_dir):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            matched = False
            for cls in classes:
                for variation in class_variations[cls]:
                    if variation.lower() in filename.lower():
                        data.append({'image_path': os.path.join(image_dir, filename), 'label': cls})
                        matched = True
                        break
                if matched:
                    break
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)
    print(f"Regenerated CSV with {len(df)} images")
    return df

# Verify that all image paths in the CSV exist
def verify_data(csv_path, img_dir):
    df = pd.read_csv(csv_path)
    print(f"Total images: {len(df)}, Classes: {df['label'].value_counts()}")
    missing = [path for path in df['image_path'] if not os.path.exists(path)]
    if missing:
        raise ValueError(f"Missing files: {missing}")

# Load and preprocess the dataset for training
def load_and_preprocess_data(csv_path, img_dir):
    df = pd.read_csv(csv_path)
    label_encoder = LabelEncoder()
    df['label_encoded'] = label_encoder.fit_transform(df['label'])
    num_classes = len(label_encoder.classes_)
    
    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
    class_weights = compute_class_weight('balanced', classes=np.unique(df['label_encoded']), y=df['label_encoded'])
    class_weight_dict = dict(enumerate(class_weights))
    def load_image(image_path, label):
        img = tf.io.read_file(image_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.cast(img, tf.float32)
        img = img / 255.0
        return img, label
    train_dataset = tf.data.Dataset.from_tensor_slices(
        (train_df['image_path'], tf.keras.utils.to_categorical(train_df['label_encoded'], num_classes))
    ).map(load_image, num_parallel_calls=tf.data.AUTOTUNE).cache().shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    val_dataset = tf.data.Dataset.from_tensor_slices(
        (val_df['image_path'], tf.keras.utils.to_categorical(val_df['label_encoded'], num_classes))
    ).map(load_image, num_parallel_calls=tf.data.AUTOTUNE).cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return train_dataset, val_dataset, num_classes, label_encoder, val_df, class_weight_dict

# Build the pretrained MobileNetV2 model
def build_pretrained_model(num_classes):
    base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    base_model.trainable = True
    for layer in base_model.layers[:-10]:
        layer.trainable = False
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax', dtype='float32')
    ])
    return model

# Plot the training history
def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    for metric in ['accuracy', 'loss']:
        plt.subplot(1, 2, 1 if metric == 'accuracy' else 2)
        plt.plot(history.history[metric], label=f'Training {metric.capitalize()}')
        plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric.capitalize()}')
        plt.title(f'Training and Validation {metric.capitalize()}')
        plt.legend()
    plt.savefig('training_history.png')
    plt.close()

# Plot the confusion matrix
def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix.png')
    plt.close()

# Main function to execute the pipeline
def main():
    df = regenerate_csv(IMG_DIR, CSV_PATH)
    if df is None:
        return
    
    verify_data(CSV_PATH, IMG_DIR)
    
    train_dataset, val_dataset, num_classes, label_encoder, val_df, class_weight_dict = load_and_preprocess_data(CSV_PATH, IMG_DIR)
    
    model = build_pretrained_model(num_classes)
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    model.build((None, IMG_HEIGHT, IMG_WIDTH, 3))
    model.summary()
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=6, min_lr=1e-6),
        CSVLogger('training_log.csv', append=True)
    ]
    
    history = model.fit(
        train_dataset, epochs=EPOCHS, validation_data=val_dataset,
        callbacks=callbacks, class_weight=class_weight_dict, verbose=1
    )
    
    val_loss, val_accuracy = model.evaluate(val_dataset)
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")
    
    model.save('model.keras')
    np.save('label_encoder_classes.npy', label_encoder.classes_)
    
    plot_training_history(history)
    
    val_images, val_labels = next(iter(val_dataset))
    predictions = model.predict(val_images)
    predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))
    true_labels = label_encoder.inverse_transform(np.argmax(val_labels, axis=1))
    print("Sample Predictions:", *[(t, p) for t, p in zip(true_labels[:10], predicted_labels[:10])], sep='\n')
    
    val_predictions = model.predict(val_dataset)
    val_pred_labels = np.argmax(val_predictions, axis=1)
    val_true_labels = np.argmax(np.concatenate([y for _, y in val_dataset]), axis=1)
    report = classification_report(val_true_labels, val_pred_labels, target_names=label_encoder.classes_)
    print("\nClassification Report:\n", report)
    
    with open('classification_report.txt', 'w') as f:
        f.write(report)
    
    plot_confusion_matrix(val_true_labels, val_pred_labels, label_encoder.classes_)

if __name__ == "__main__":
    main()

Num GPUs Available:  1
Regenerated CSV with 12335 images
Total images: 12335, Classes: label
living room    2621
dining room    2605
bed            2445
bath           2430
kitchen        2234
Name: count, dtype: int64
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_11  (None, 1280)             0         
 66 (GlobalAveragePooling2D)                                     
                                                                 
 dense_10 (Dense)            (None, 512)               655872    
                                                                 
 dropout_25 (Dropout)        (None, 512)               0         
                                 

2025-05-21 05:25:49,475 - WARNING - Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0969s vs `on_train_batch_end` time: 0.1923s). Check your callbacks.


124/124 [==============================] - 34s 131ms/step - loss: 0.5803 - accuracy: 0.7937 - val_loss: 12.6877 - val_accuracy: 0.2623 - lr: 0.0010
Epoch 2/140
124/124 [==============================] - 18s 147ms/step - loss: 0.3727 - accuracy: 0.8695 - val_loss: 14.7284 - val_accuracy: 0.3624 - lr: 0.0010
Epoch 3/140
124/124 [==============================] - 7s 55ms/step - loss: 0.2914 - accuracy: 0.8999 - val_loss: 13.5066 - val_accuracy: 0.3656 - lr: 0.0010
Epoch 4/140
124/124 [==============================] - 7s 55ms/step - loss: 0.2385 - accuracy: 0.9153 - val_loss: 7.4277 - val_accuracy: 0.5116 - lr: 0.0010
Epoch 5/140
124/124 [==============================] - 7s 56ms/step - loss: 0.1879 - accuracy: 0.9336 - val_loss: 10.0992 - val_accuracy: 0.4702 - lr: 0.0010
Epoch 6/140
124/124 [==============================] - 7s 56ms/step - loss: 0.1692 - accuracy: 0.9408 - val_loss: 9.3952 - val_accuracy: 0.4852 - lr: 0.0010
Epoch 7/140
124/124 [==============================] - 7s 54ms